# Creating Bike and Pedestrian Road Links

For environment see: https://github.com/gboeing/osmnx 

    conda config --prepend channels conda-forge
    
    conda create -n ox --strict-channel-priority osmnx
    
* also include nb_conda, nb_conda_kernels, and ipykernel

In [1]:
import geopandas as gpd
import osmnx
import pandas as pd
import networkx as nx

In [ ]:
#import TAZs
tazs = gpd.read_file('C:/Users/phoebe/Bike_Ped_Skims_OSM/candidate_CTPS_TAZ_STATEWIDE_2019.geojson')
#set the projected crs
tazs = tazs.to_crs('EPSG:26986')
#boundaries for TAZ
states = tazs.dissolve(by='state')

#update the attributes retrieved from OSM in settings.py
useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 
                    'surface', 'smoothness','cycleway', 'footway', 'sidewalk']
osmnx.utils.config(useful_tags_way = useful_tags_path)

#grab from OSM the bike and pedestrian networks for the MPO
#WARNING: Takes 48 hours to run (for both bike and ped together)
bike_net = osmnx.graph_from_polygon(states.to_crs('EPSG:4326').unary_union, network_type='bike')
ped_net = osmnx.graph_from_polygon(states.to_crs('EPSG:4326').unary_union, network_type='walk')

#turn to gdf to see the attributes (tags) on the links
bike_edges = osmnx.utils_graph.graph_to_gdfs(bike_net, nodes = False, edges = True,
                                             node_geometry = False, fill_edge_geometry = False)
#turn to gdf to see the attributes (tags) on the links
ped_edges = osmnx.utils_graph.graph_to_gdfs(ped_net, nodes = False, edges = True,
                                             node_geometry = False, fill_edge_geometry = False)

#export to shp
bike_edges.to_file("bike_edges.shp")
ped_edges.to_file("ped_edges.shp")